In [3]:
import re
import os 
import datetime
from math import log
import csv
import pprint
import math
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
import pandas as pd
from pandas.io.parsers import read_csv
pd.set_option('max_colwidth',120)
from elasticsearch import Elasticsearch, helpers
from elasticsearch.helpers import scan
#from __future__ import division:q
import urllib

%matplotlib inline

pp = pprint.PrettyPrinter()

In [2]:
#bashcommand='pwd'

In [4]:
es = Elasticsearch([{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=20)

In [5]:
list=['jobs_archive_2017-02-10','jobs_archive_2017-02-11','jobs_archive_2017-02-12','jobs_archive_2017-02-13','jobs_archive_2017-02-14']
#print (list)
indices = es.cat.indices(index=list, h="index", request_timeout=10).split('\n')
indices = sorted(indices)
indices = [x.replace(" ", "") for x in indices if x != '']
 

In [11]:
print (indices)

['jobs_archive_2017-02-10', 'jobs_archive_2017-02-11', 'jobs_archive_2017-02-12', 'jobs_archive_2017-02-13', 'jobs_archive_2017-02-14']


In [11]:
ss={
 "query": { "wildcard": { "computingsite": "*ROMANIA*" } 
}, "_source": ["pandaid","batchid","computingsite","computingelement","corecount","creationtime","starttime","endtime",
   "cpuconsumptiontime","wall_time","modificationhost","maxrss","maxpss","jobstatus","inputfilebytes",
               "outputfilebytes","timeGetJob","timeSetup","timeStageIn","timeExe","timeStageOut","wall_time","ninputdatafiles","noutputdatafiles"]
#    ,"size": 5
    }



#ss= {"query": { "match_all": {} },"size": 10000} 
print (ss)



{'query': {'wildcard': {'computingsite': '*ROMANIA*'}}, '_source': ['pandaid', 'batchid', 'computingsite', 'computingelement', 'corecount', 'creationtime', 'starttime', 'endtime', 'cpuconsumptiontime', 'wall_time', 'modificationhost', 'maxrss', 'maxpss', 'jobstatus', 'inputfilebytes', 'outputfilebytes', 'timeGetJob', 'timeSetup', 'timeStageIn', 'timeExe', 'timeStageOut', 'wall_time', 'ninputdatafiles', 'noutputdatafiles']}


In [7]:
#res = es.search(index='jobs_archive_2016-06-26', body=ss, filter_path=['hits.hits._source'],request_timeout=12000)

In [12]:
res=es.search(index=indices, body=ss,request_timeout=12000)
#print (res)
len(res)
#print (res)

4

In [9]:
 #res.get('hits').get('hits')

In [13]:
scroll=scan(es,index=indices, query=ss,scroll=u'5m', raise_on_error=True, preserve_order=False, size=10000,request_timeout=700000)

In [14]:
c=1
list2=[]
for name in scroll:
 a=name.get('_source')
 list2.append(a)
 c=c+1
print (c)
type(list2)
dfu = pd.DataFrame(list2)

dfu.to_csv('./romania_feb2017.csv')

62123


In [15]:
dfu.count()

batchid               59976
computingelement      62122
computingsite         62122
corecount             61418
cpuconsumptiontime    62122
creationtime          62122
endtime               62122
inputfilebytes        60867
jobstatus             62122
maxpss                59685
maxrss                59685
modificationhost      62122
ninputdatafiles       60867
noutputdatafiles      62122
outputfilebytes       62122
pandaid               62122
starttime             59978
timeExe               62122
timeGetJob            62122
timeSetup             62122
timeStageIn           62122
timeStageOut          62122
wall_time             62122
dtype: int64

In [ ]:
dfu['read']=dfu['inputfilebytes']/dfu['timeStageIn']/1024
dfu['write']=dfu['outputfilebytes']/dfu['timeStageOut']/1024
#dfu['starttime'] = dfu['starttime'].map(lambda x: pd.to_datetime(x,format="%Y-%m-%d %H:%M:%S"))
dfu.tail()

In [ ]:
pp2=dfu.dropna()
pp2.count()

In [ ]:
ppp=pp2.groupby([pd.Grouper(freq='5min',key='starttime')]).read.sum()/1024
ppp.plot()
pp2.inputfilebytes.sum()/1024/1024/1024


In [ ]:
dfu[pd.isnull(dfu).any(axis=1)]

In [ ]:


#rint (res.get('hits').get('hits'))
#list.head()
#res[0]
#res['total']

In [16]:
 dfu.groupby(['computingsite']).count()

,batchid,computingsite,corecount,cpuconsumptiontime,creationtime,endtime,inputfilebytes,jobstatus,maxpss,maxrss,...,noutputdatafiles,outputfilebytes,pandaid,starttime,timeExe,timeGetJob,timeSetup,timeStageIn,timeStageOut,wall_time
computingelement,,,,,,,,,,,,,,,,,,,,,
ANALY_ROMANIA02,21567,21567,21200,21567,21567,21567,20983,21567,21562,21562,...,21567,21567,21567,21567,21567,21567,21567,21567,21567,21567
ANALY_ROMANIA07,11991,11994,11690,11994,11994,11994,11474,11994,11769,11769,...,11994,11994,11994,11993,11994,11994,11994,11994,11994,11994
RO-02-NIPNE-tbat03-atlas-lcgpbs,3579,3579,3575,3579,3579,3579,3575,3579,3579,3579,...,3579,3579,3579,3579,3579,3579,3579,3579,3579,3579
RO-02-NIPNE-tbat03-atlas-lcgpbs_MCORE,1999,1999,1994,1999,1999,1999,1994,1999,1999,1999,...,1999,1999,1999,1999,1999,1999,1999,1999,1999,1999
RO-07-NIPNE-creamce03-atlas-lcgpbs,7691,7691,7686,7691,7691,7691,7686,7691,7691,7691,...,7691,7691,7691,7691,7691,7691,7691,7691,7691,7691
RO-07-NIPNE-creamce07-atlasmcore,1438,1438,1438,1438,1438,1438,1438,1438,1400,1400,...,1438,1438,1438,1438,1438,1438,1438,1438,1438,1438
RO-14-ITIM-cream-atlas-pbs,1060,1060,1058,1060,1060,1060,1058,1060,1060,1060,...,1060,1060,1060,1060,1060,1060,1060,1060,1060,1060
RO-14-ITIM_DISKLESS,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
RO-16-UAIC-cream-grid-atlas-lcgpbs,1534,1534,1529,1534,1534,1534,1529,1534,1534,1534,...,1534,1534,1534,1534,1534,1534,1534,1534,1534,1534


In [ ]:
#b=dfu.maxpss/1024/1024
#b.hist(bins=100)

In [ ]:
#dfu.memory_usage()

In [ ]:
dfu.to_csv('./romania_16_feb2017.csv')